In [ ]:
# default_exp train

# Train

> API details.

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib as mpl
%matplotlib inline

In [ ]:
#export
import warnings
import re
from functools import partial

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

import pytorch_lightning as pl
from pytorch_lightning.core import LightningModule
from pytorch_lightning.metrics import functional as FM

In [ ]:
#export
from isic.dataset import SkinDataModule, from_label_idx_to_key
from isic.layers import LabelSmoothingCrossEntropy
from isic.callback.hyperlogger import HyperparamsLogger
from isic.callback.logtable import LogTableMetricsCallback
from isic.callback.mixup import MixupDict
from isic.callback.cutmix import CutmixDict
from isic.callback.freeze import FreezeCallback, UnfreezeCallback
from isic.utils.core import reduce_loss, generate_val_steps
from isic.utils.model import apply_init, get_bias_batchnorm_params, apply_leaf, check_attrib_module, create_body, create_head, lr_find, freeze, unfreeze, log_metrics_per_key
from isic.model import BaselineModel, Model

In [ ]:
message_formater = "You have set {0} number of classes if different from predicted {0} and target {0} number of classes"
warnings.filterwarnings("ignore", message_formater.format("(.*)"), category=UserWarning)

In [ ]:
dm = SkinDataModule()
dm.prepare_data()
dm.setup('fit')

In [ ]:
F_EPOCHS = 1
U_EPOCHS = 1
LR = 1e-2

# Baseline

In [ ]:
model = BaselineModel('resnet18')
trainer = pl.Trainer(fast_dev_run=True, callbacks=[LogTableMetricsCallback()])
trainer.fit(model, dm)

Running in fast_dev_run mode: will run a full train, val and test loop using a single batch
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
C:\Users\bomco\miniconda3\envs\devtorch\lib\site-packages\pytorch_lightning\utilities\distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)

  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 11 M  


C:\Users\bomco\miniconda3\envs\devtorch\lib\site-packages\pytorch_lightning\utilities\distributed.py:37: RuntimeWarning: The metric you returned None must be a `torch.Tensor` instance, checkpoint not saved HINT: what is the value of loss in validation_epoch_end()?
  warnings.warn(*args, **kwargs)
C:\Users\bomco\miniconda3\envs\devtorch\lib\site-packages\pytorch_lightning\utilities\distributed.py:37: RuntimeWarning: Can save best model only with loss available, skipping.
  warnings.warn(*args, **kwargs)


{'train_loss': 2.334678888320923, 'epoch': 0}
{'val_acc': 0.0, 'val_precision': 0.0, 'val_recall': 0.0, 'F1': 0.0, 'val_precision_akiec': 0.0, 'val_precision_bcc': 0.0, 'val_precision_bkl': 0.0, 'val_precision_df': 0.0, 'val_precision_nv': 0.0, 'val_precision_mel': 0.0, 'val_precision_vasc': 0.0, 'val_recall_akiec': 0.0, 'val_recall_bcc': 0.0, 'val_recall_bkl': 0.0, 'val_recall_df': 0.0, 'val_recall_nv': 0.0, 'val_recall_mel': 0.0, 'val_recall_vasc': 0.0, 'val_loss': 21.035709381103516}
{'val_early_stop_on': None, 'val_checkpoint_on': None, 'checkpoint_on': tensor(2.3347)}


epoch,train_loss,val_acc,val_precision,val_recall,F1,val_precision_akiec,val_precision_bcc,val_precision_bkl,val_precision_df,val_precision_nv,val_precision_mel,val_precision_vasc,val_recall_akiec,val_recall_bcc,val_recall_bkl,val_recall_df,val_recall_nv,val_recall_mel,val_recall_vasc,val_loss,time
1,2.335,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,21.035709,00:38


Saving latest checkpoint..


1

In [ ]:
dm.setup('test')
a = trainer.test(model, dm.val_dataloader())

/home/termanteus/miniconda3/envs/devtorch/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)


hehe
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'F1': tensor(0.0571),
 'val_acc': tensor(0.2500),
 'val_precision': tensor(0.1429),
 'val_precision_akiec': tensor(0.),
 'val_precision_bcc': tensor(0.),
 'val_precision_bkl': tensor(1.),
 'val_precision_df': tensor(0.),
 'val_precision_mel': tensor(0.),
 'val_precision_nv': tensor(0.),
 'val_precision_vasc': tensor(0.),
 'val_recal_akiec': tensor(0.),
 'val_recal_bcc': tensor(0.),
 'val_recal_bkl': tensor(0.2500),
 'val_recal_df': tensor(0.),
 'val_recal_mel': tensor(0.),
 'val_recal_nv': tensor(0.),
 'val_recal_vasc': tensor(0.),
 'val_recall': tensor(0.0357)}
--------------------------------------------------------------------------------



In [ ]:
torch.load('preds.pt').shape

torch.Size([64])

# Real

In [ ]:
# init model
model = Model(LR, arch='resnet18')

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /home/termanteus/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [ ]:
check_attrib_module(model)

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
['weight-requires_grad-False']
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
['weight-requires_grad-True', 'weight-skip_wd-True', 'bias-requires_grad-True', 'bias-skip_wd-True']
ReLU(inplace=True)
[]
MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
[]
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
['weight-requires_grad-False']
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
['weight-requires_grad-True', 'weight-skip_wd-True', 'bias-requires_grad-True', 'bias-skip_wd-True']
ReLU(inplace=True)
[]
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
['weight-requires_grad-False']
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
['weight-requires_grad-True', 'weight-skip_wd-True', 'bias-requires_grad-True', 'bias-skip_wd-True']
Conv2d(64

In [ ]:
lr_find(model, dm,lr_find=False,verbose=True)

Running in fast_dev_run mode: will run a full train, val and test loop using a single batch
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
C:\Users\bomco\miniconda3\envs\devtorch\lib\site-packages\pytorch_lightning\utilities\distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)

  | Name      | Type                       | Params
---------------------------------------------------------
0 | model     | Sequential                 | 11 M  
1 | m_bacc    | BalancedAccuracy           | 0     
2 | loss_func | LabelSmoothingCrossEntropy | 0     


override_called
[0.01, 0.01, 0.01, 0.01, 0.01, 0.01]


C:\Users\bomco\miniconda3\envs\devtorch\lib\site-packages\sklearn\metrics\_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
Saving latest checkpoint..



Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01
    weight_decay: 0.0

Parameter Group 1
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01
    weight_decay: 0.0

Parameter Group 2
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01
    weight_decay: 0.0

Parameter Group 3
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01
    weight_decay: 0.0

Parameter Group 4
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01
    weight_decay: 0.0

Parameter Group 5
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01
    weight_decay: 0.0
)
******************************Check requires_grad/ skip_wd******************************
Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
['weight-requires_grad-False']
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
['weight-requires_grad-True', 'weight-skip

In [ ]:
cbs = [LogTableMetricsCallback(), HyperparamsLogger()]
trainer = fit_one_cycle(F_EPOCHS, model, dm, max_lr=LR, callbacks=cbs, fast_dev_run=False, limit_val_batches=0, limit_train_batches=0.01)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [ ]:
unfreeze(model, 3)

In [ ]:
# Unfreeze training
trainer = fit_one_cycle(callbacks=cbs, fast_dev_run=False, limit_val_batches=0, limit_train_batches=0.01)

Running in fast_dev_run mode: will run a full train, val and test loop using a single batch
GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [ ]:
trainer.fit(model, dm)


  | Name      | Type             | Params
-----------------------------------------------
0 | model     | Sequential       | 25 M  
1 | loss_func | CrossEntropyLoss | 0     


override_called
wtf


epoch,train_loss,train_acc,val_loss,val_acc
1,3.256,0.156250,4.279556,0.000000


Saving latest checkpoint..


1

# Tensorboard

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir=lightning_logs/

Reusing TensorBoard on port 6006 (pid 4636), started 4 days, 18:33:26 ago. (Use '!kill 4636' to kill it.)